-: Mongo - Neo User story :- Data Bases as Search Engines :-   

As a End user, I have a searcing word (or) sentence, based on my input, I want
to know, the mobile which is more near to my thinking & who is the manufacturer produced it and which market area..

Like Google search..

Hint :- Please dont forget to start "Complaints & Review Graph" in Neo4J
Hint : -If it is not working, please create index for this collection in mongo Shell
db.Reviews_Collection.createIndex({review_body:"text"})

In [1]:
  ############################# Importing all req. libreries ############################################
from bson.son import SON
from py2neo import Graph,Node,Relationship
from pymongo import MongoClient
import pandas as pd
g = Graph()
client = MongoClient("mongodb://localhost:27017")
database = client["Open_Source_DB"]
collection = database["Reviews_Collection"]


In [6]:
  ############################# Input word (or) sentence  ############################################
inp = "\"Hello\""
   ############################# Mongo Pipeline  ############################################
pipeline = [
    {
        u"$match": {
            u"$text": {
                u"$search": inp
            }
        }
    }, 
    {
        u"$project": {
            u"Ranking": {
                u"$meta": u"textScore"
            },
            u"Review_ID": u"$Review_ID",
            u"review_body": u"$review_body",
            u"_id": 0.0
        }
    }, 
    {
        u"$sort": SON([ (u"Ranking", -1) ])
    }
]

cursor = collection.aggregate(
    pipeline, 
    allowDiskUse = False
)

out = []
try:
    for doc in cursor:
        out.append(doc['Review_ID'])
        print(doc)
        print("-----------------------------------------------------------------------------------------")
finally:
    client.close()

{'Ranking': 1.1, 'Review_ID': 'review_27663', 'review_body': 'Hello'}
-----------------------------------------------------------------------------------------
{'Ranking': 0.9187500000000001, 'Review_ID': 'review_71548', 'review_body': 'I have a Samsung Galaxy Note 3 that has several intermittent problems: 1. The phone does not ring. 2. It goes on mute for no reason or the same effect as mute as the listener cannot hear me when I speak. Listener thinks the call has been dropped and hangs up. Most common thing I hear on my phone are “Hello?, Hello? , Hello? And “Are you there.” 3. If I put the phone in speaker the listener hears an echo whenever they speak. It’s is so annoying that the listeners insist I take the call of the speaker function. I have never had problems with any other phone I have owned and I have used all the service providers at one time or another and have lived at the same location.'}
------------------------------------------------------------------------------------

In [7]:
len(out)

100

In [8]:
out = out[:10]

for rev in out:
    df = g.run('''MATCH (r:Review)-[:REVIEW_ORDER]->(o:Order)-[:PURCHASED_DETAILS]->(m:Mobile)
MATCH (o)-[:MANUFACTURED_BY]->(ma:Manufacturer)-[:LOCATED_IN]->(c:Country)
WHERE r.Review_Id = "''' + rev + '''"
RETURN m.Mobile_Name AS Mobile, ma.Manuf_Id AS ManufactuerID,c.Country AS Country''').to_data_frame()
    print(df[:1].to_string(index=False,header=False))

 Samsung M2520 Beat Techno  f7496d659ca9fdaf323c0aae84176632  Spain
 Samsung M350 Seek  d2e753bb80b7d4faa77483ed00edc8ca  Germany
 Samsung Galaxy A90_5G  f9ec7093df3a7b346b7bcf7864069ca3  Germany
 Samsung Messenger Touch  f7496d659ca9fdaf323c0aae84176632  Spain
 Samsung Shark 3  8bd0e3abda539b9479c4b44a691be1ec  USA
 Samsung Epic 5G  f7496d659ca9fdaf323c0aae84176632  Spain
 Samsung S3650W Corby  f9ec7093df3a7b346b7bcf7864069ca3  Germany
 Samsung Messenger Touch  d2e753bb80b7d4faa77483ed00edc8ca  Germany
 Samsung Fascinate  2a1348e9addc1af5aaa619b1a3679d6b  USA
 Samsung Messenger Touch  d2e753bb80b7d4faa77483ed00edc8ca  Germany
